In [9]:
pip install PyOpenGL PyOpenGL_accelerate pygame

You should consider upgrading via the 'c:\Users\slava\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [10]:
import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
import random
import sys
import math

In [ ]:
class Gameplay3D:
    def __init__(self):
        self.SCREEN_WIDTH = 1280
        self.SCREEN_HEIGHT = 720
        self.FOV = 45
        self.aspect_ratio = self.SCREEN_WIDTH / self.SCREEN_HEIGHT
        self.near_plane = 0.1
        self.far_plane = 50.0
        self.camera_x = 0
        self.camera_y = 1
        self.camera_z = 5
        self.camera_speed = 0.1
        self.yaw = 0
        self.pitch = 0
        self.textures = {}
        self.load_textures()

    def load_textures(self):
        # Загрузка текстур из папки gfx
        self.textures['floor'] = self.load_texture('gfx/brick.png')
        self.textures['enemy'] = self.load_texture('gfx/enemy_texture.png')

    def load_texture(self, filename):
        texture_surface = pygame.image.load(filename)
        texture_data = pygame.image.tostring(texture_surface, "RGB", True)
        width, height = texture_surface.get_size()

        # Генерация и настройка текстуры в OpenGL
        texture_id = glGenTextures(1)
        glBindTexture(GL_TEXTURE_2D, texture_id)
        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, texture_data)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
        return texture_id

    def init_3d(self):
        glEnable(GL_DEPTH_TEST)
        glEnable(GL_TEXTURE_2D)  # Включаем текстуры
        glMatrixMode(GL_PROJECTION)
        gluPerspective(self.FOV, self.aspect_ratio, self.near_plane, self.far_plane)
        glMatrixMode(GL_MODELVIEW)

    def handle_keys(self, keys):
        if keys[K_w]:
            self.camera_x -= self.camera_speed * math.sin(math.radians(self.yaw))
            self.camera_z -= self.camera_speed * math.cos(math.radians(self.yaw))
        if keys[K_s]:
            self.camera_x += self.camera_speed * math.sin(math.radians(self.yaw))
            self.camera_z += self.camera_speed * math.cos(math.radians(self.yaw))
        if keys[K_a]:
            self.camera_x -= self.camera_speed * math.cos(math.radians(self.yaw))
            self.camera_z += self.camera_speed * math.sin(math.radians(self.yaw))
        if keys[K_d]:
            self.camera_x += self.camera_speed * math.cos(math.radians(self.yaw))
            self.camera_z -= self.camera_speed * math.sin(math.radians(self.yaw))

    def update_camera(self):
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        glLoadIdentity()
        
        gluLookAt(
            self.camera_x, self.camera_y, self.camera_z,
            self.camera_x + math.sin(math.radians(self.yaw)), self.camera_y, self.camera_z + math.cos(math.radians(self.yaw)),
            0, 1, 0
        )

    def draw_textured_cube(self, x, y, z, texture_id):
        glBindTexture(GL_TEXTURE_2D, texture_id)
        glPushMatrix()
        glTranslatef(x, y, z)
        glBegin(GL_QUADS)

        # Верхняя грань
        glTexCoord2f(0, 0); glVertex3f(1, 1, -1)
        glTexCoord2f(1, 0); glVertex3f(-1, 1, -1)
        glTexCoord2f(1, 1); glVertex3f(-1, 1, 1)
        glTexCoord2f(0, 1); glVertex3f(1, 1, 1)

        # Нижняя грань
        glTexCoord2f(0, 0); glVertex3f(1, -1, 1)
        glTexCoord2f(1, 0); glVertex3f(-1, -1, 1)
        glTexCoord2f(1, 1); glVertex3f(-1, -1, -1)
        glTexCoord2f(0, 1); glVertex3f(1, -1, -1)

        # Передняя грань
        glTexCoord2f(0, 0); glVertex3f(1, 1, 1)
        glTexCoord2f(1, 0); glVertex3f(-1, 1, 1)
        glTexCoord2f(1, 1); glVertex3f(-1, -1, 1)
        glTexCoord2f(0, 1); glVertex3f(1, -1, 1)

        # Задняя грань
        glTexCoord2f(0, 0); glVertex3f(1, -1, -1)
        glTexCoord2f(1, 0); glVertex3f(-1, -1, -1)
        glTexCoord2f(1, 1); glVertex3f(-1, 1, -1)
        glTexCoord2f(0, 1); glVertex3f(1, 1, -1)

        glEnd()
        glPopMatrix()

NameError: name 'sin' is not defined

: 

In [ ]:
if __name__ == '__main__':
    pygame.init()
    pygame.display.set_mode((1280, 720), DOUBLEBUF | OPENGL)
    pygame.display.set_caption("3D Dungeon Sandbox")
    gameplay = Gameplay3D()
    gameplay.init_3d()
    clock = pygame.time.Clock()

    # Позиции текстурных кубов
    cube_positions = [(random.uniform(-10, 10), 0, random.uniform(-10, 10)) for _ in range(10)]

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        keys = pygame.key.get_pressed()
        gameplay.handle_keys(keys)
        gameplay.update_camera()
        
        # Отрисовка текстурных кубов
        for pos in cube_positions:
            gameplay.draw_textured_cube(*pos, gameplay.textures['floor'])

        pygame.display.flip()
        clock.tick(60)

    pygame.quit()